## 1. 문제 제기
* 주제 : 태양광 발전량 예측을 통해 휴전점검일과 관리 필요성 알림

## 2. 데이터 구하기
* 1. 기상자료개방포털 : https://data.kma.go.kr/cmmn/main.do
* 2. DNE Solar : http://www.qs2200.co.kr/SrMain/SM020.aspx

## 3. 타겟변수 설정
* 타겟변수명 : 발전량
* 타겟변수값 : 실수

## 4. 데이터 처리
* 4-1 데이터 불러오기
* 4-2 ID 변수 설정
* 4-3 타겟변수 생성
* 4-4 기타 변수 데이터 처리
* 결과 파일 

### 4-1. 데이터 불러오기

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('태양광_데이터_통합.csv', encoding="cp949")

df.head(3)

,날짜,발전기명,최고기온(℃),최저기온(℃),강수량(mm),평균풍속(m/s),평균 전운량(1/10),발전량(kWh)
0,2021-06-01,광양항광양냉장태양광,28.8,15.4,NaN,0.9,4.9,4.244454
1,2021-06-02,광양항광양냉장태양광,28.6,17.7,NaN,0.8,7.9,3.248909
2,2021-06-03,광양항광양냉장태양광,20.6,17.9,22.0,1.2,9.6,0.729910


In [4]:
# 데이터프레임 df행과 열 차원 확인
df.shape

(4887, 8)

In [5]:
# 데이터프레임 df 정보(info) 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4887 entries, 0 to 4886
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   날짜            4887 non-null   object 
 1   발전기명          4887 non-null   object 
 2   최고기온(℃)       4887 non-null   float64
 3   최저기온(℃)       4887 non-null   float64
 4   강수량(mm)       1744 non-null   float64
 5   평균풍속(m/s)     4886 non-null   float64
 6   평균 전운량(1/10)  4885 non-null   float64
 7   발전량(kWh)      4887 non-null   float64
dtypes: float64(6), object(2)
memory usage: 305.6+ KB


### 4-2 ID 변수 설정
* ID 변수 데이터 타입
* 결측값
* 중복 여부
* 숫자가 아닌 값

In [6]:
# ID 변수인 id  데이터 타입 확인
df['일강수량(mm)'].dtype

KeyError: '일강수량(mm)'

In [ ]:
# ID 변수인 id 값의 결측값 확인
df['일강수량(mm)'].isnull().sum()

In [7]:
# ID 변수인 id 값의 중복 여부를 체크하기 위해 유일한(unique) 값의 개수 체크
n = len(pd.unique(df['일강수량(mm)']))  # pd.unique는 유일한 변수값을 출력하고 len은 값의 개수를 카운트 한다.
n

145

In [8]:
#  숫자형태가 아닌 SERIALNO의 갯수를 세튼 파이썬 코드
mask = pd.to_numeric(df['일강수량(mm)'], errors='coerce').isna()
a=mask.sum()
a

641

### 4-3. 타겟 변수 생성

In [9]:
# 타켓변수 데이터 타입 확인
df['발전량'].dtype

dtype('float64')

In [10]:
# 타켓변수 결측값 확인.
df['발전량'].isnull().sum()

0

In [11]:
# 타켓변수 클래스(0, 1값) 갯수 확인
df['발전량'].value_counts(dropna=False)

0.0      6
376.6    3
528.8    3
341.6    3
437.2    3
        ..
349.3    1
473.9    1
248.0    1
82.7     1
316.9    1
Name: 발전량, Length: 864, dtype: int64

In [12]:
df['발전량'].value_counts(dropna=False, normalize=True)

0.0      0.006363
376.6    0.003181
528.8    0.003181
341.6    0.003181
437.2    0.003181
           ...   
349.3    0.001060
473.9    0.001060
248.0    0.001060
82.7     0.001060
316.9    0.001060
Name: 발전량, Length: 864, dtype: float64

## 6. 머신러닝 모델 실행
* 머신러닝 모델 수립
* 데이터 추가 처리(문자형 데이터)
* 데이터 분할 및 대체
* 결과 파일 저장

### 6-2. 데이터 추가 처리

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('태양광_데이터.csv')
df.shape

(943, 8)

In [3]:
# 데이터프레임 변수 데이터타입 확인.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   일자            943 non-null    object 
 1   지점명           943 non-null    object 
 2   최저기온(°C)      943 non-null    float64
 3   최고기온(°C)      943 non-null    float64
 4   일강수량(mm)      302 non-null    float64
 5   평균 풍속(m/s)    942 non-null    float64
 6   평균 전운량(1/10)  941 non-null    float64
 7   발전량           943 non-null    float64
dtypes: float64(6), object(2)
memory usage: 59.1+ KB


In [4]:
# 컬럼명(변수명) 구하기 
df.columns

Index(['일자', '지점명', '최저기온(°C)', '최고기온(°C)', '일강수량(mm)', '평균 풍속(m/s)',
       '평균 전운량(1/10)', '발전량'],
      dtype='object')

In [5]:
# 컬럼명(변수명) 구하기 
list(df.columns)

['일자',
 '지점명',
 '최저기온(°C)',
 '최고기온(°C)',
 '일강수량(mm)',
 '평균 풍속(m/s)',
 '평균 전운량(1/10)',
 '발전량']

## 6.3 데이터 분할 및 대체

In [8]:
data = df[[
    '최저기온(°C)',
    '최고기온(°C)',
    '일강수량(mm)',
    '평균 풍속(m/s)',
    '평균 전운량(1/10)'
]] # 타켓변수를 제외한 변수만 data에 저장

target = df['발전량'].div(99)              # 타켓변수만 target에 저장.
data = data.fillna(0)
data.info(), target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   최저기온(°C)      943 non-null    float64
 1   최고기온(°C)      943 non-null    float64
 2   일강수량(mm)      943 non-null    float64
 3   평균 풍속(m/s)    943 non-null    float64
 4   평균 전운량(1/10)  943 non-null    float64
dtypes: float64(5)
memory usage: 37.0 KB


(None,
 0      1.292929
 1      0.770707
 2      1.485859
 3      0.798990
 4      1.966667
          ...   
 938    5.275758
 939    4.516162
 940    5.872727
 941    3.627273
 942    3.201010
 Name: 발전량, Length: 943, dtype: float64)

In [18]:
target.shape

(943,)

In [19]:
df['발전량'].value_counts(dropna=False)

0.0      6
376.6    3
528.8    3
341.6    3
437.2    3
        ..
349.3    1
473.9    1
248.0    1
82.7     1
316.9    1
Name: 발전량, Length: 864, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
data, target, test_size = 0.2, random_state=42)

print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

X_train shape:  (754, 5)
X_test shape:  (189, 5)


In [21]:
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

y_train shape:  (754,)
y_test shape:  (189,)


In [22]:
y_train.value_counts(normalize=True)

0.0      0.005305
437.2    0.003979
528.8    0.003979
376.6    0.003979
431.6    0.003979
           ...   
79.1     0.001326
214.4    0.001326
418.6    0.001326
61.8     0.001326
433.0    0.001326
Name: 발전량, Length: 702, dtype: float64

### ML 모델 구현해 주세요

In [23]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor(random_state=0)
tree.fit(X_train, y_train)

print("Accuracy on training set:{:.5f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set:{:.5f}".format(tree.score(X_test, y_test)))

Accuracy on training set:1.00000
Accuracy on test set:0.36522


In [24]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth':[2 ,4 ,6, 8, 10, 12],
    'min_samples_leaf':[2 ,4, 6, 8, 12, 18],
    'min_samples_split':[2 ,4, 6, 8, 16, 20]
}

stroke_model_RFR = RandomForestRegressor(n_jobs=-1)

stroke_model_RFR_grid_cv = GridSearchCV(stroke_model_RFR, param_grid = params, cv=2, n_jobs=-1)

stroke_model_RFR_grid_cv.fit(X_train, y_train)

print("Accuracy on training set:{:.5f}".format(stroke_model_RFR_grid_cv.score(X_train, y_train)))
print("Accuracy on test set:{:.5f}".format(stroke_model_RFR_grid_cv.score(X_test, y_test)))
print('최적의 하이퍼 파라미터 :',stroke_model_RFR_grid_cv.best_params_)
print('최적의 예측 정확도 :',stroke_model_RFR_grid_cv.best_score_)

Accuracy on training set:0.76699
Accuracy on test set:0.69701
최적의 하이퍼 파라미터 : {'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 8, 'n_estimators': 100}
최적의 예측 정확도 : 0.6157721290256067


In [25]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth':[2 ,4 ,6, 8, 10, 12],
    'min_samples_leaf':[2 ,4, 6, 8, 12, 18],
    'min_samples_split':[2 ,4, 6, 8, 16, 20]
}

stroke_model_Gradient = GradientBoostingRegressor()

stroke_model_Gradient_grid_cv = GridSearchCV(stroke_model_Gradient, param_grid = params, cv=2, n_jobs=-1)

stroke_model_Gradient_grid_cv.fit(X_train, y_train)

print("Accuracy on training set:{:.5f}".format(stroke_model_Gradient_grid_cv.score(X_train, y_train)))
print("Accuracy on test set:{:.5f}".format(stroke_model_Gradient_grid_cv.score(X_test, y_test)))
print('최적의 하이퍼 파라미터 :',stroke_model_Gradient_grid_cv.best_params_)
print('최적의 예측 정확도 :',stroke_model_Gradient_grid_cv.best_score_)

Accuracy on training set:0.73688
Accuracy on test set:0.68683
최적의 하이퍼 파라미터 : {'max_depth': 2, 'min_samples_leaf': 18, 'min_samples_split': 2, 'n_estimators': 100}
최적의 예측 정확도 : 0.640781094532175


In [26]:
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth':[2 ,4 ,6, 8, 10, 12],
    'min_samples_leaf':[2 ,4, 6, 8, 12, 18],
    'min_samples_split':[2 ,4, 6, 8, 16, 20]
}

stroke_model_LGBM = LGBMRegressor()

stroke_model_LGBM_grid_cv = GridSearchCV(stroke_model_LGBM, param_grid = params, cv=2, n_jobs=-1)

stroke_model_LGBM_grid_cv.fit(X_train, y_train)                              
                              
print("Accuracy on training set:{:.5f}".format(stroke_model_LGBM_grid_cv.score(X_train, y_train)))
print("Accuracy on test set:{:.5f}".format(stroke_model_LGBM_grid_cv.score(X_test, y_test)))
print('최적의 하이퍼 파라미터 :',stroke_model_LGBM_grid_cv.best_params_)
print('최적의 예측 정확도 :',stroke_model_LGBM_grid_cv.best_score_)

[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_samples_leaf=18. Current value: min_data_in_leaf=18
[LightGBM] [Warning] Unknown parameter: min_samples_split
Accuracy on training set:0.73686
Accuracy on test set:0.68399
최적의 하이퍼 파라미터 : {'max_depth': 2, 'min_samples_leaf': 18, 'min_samples_split': 2, 'n_estimators': 100}
최적의 예측 정확도 : 0.6389068850679018
